<a href="https://colab.research.google.com/github/jallenrobern/CCPGLANG/blob/main/PAN_Syntax_Analyzer_Cook%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🥘 PAN Analyzer — A Cooking-Inspired Programming Language (Cook++)**


---









## **📖 Project Overview**

**PAN Analyzer** is a lexical and syntax analyzer for Cook++, a recipe-inspired programming language designed to teach basic programming concepts through cooking metaphors.
The language transforms everyday cooking instructions like mix, chop, and bake into structured programming statements that represent functions, control flow, and expressions.

This project demonstrates how a domain-specific language (DSL) can make programming more intuitive and engaging for beginners.


---



## **🎯 Objectives**


1.   To design a simple yet expressive programming language using real-world cooking analogies.
2.   To implement a lexical analyzer and parser for Cook++ using Python.


1.   To apply BNF grammar and demonstrate language structure and syntax validation.
2.   To provide test coverage using multiple sample Cook++ programs.


---











In [1]:
!pip install flask flask-cors pyngrok

In [2]:
from pyngrok import ngrok

# 🔐 Set your auth token
ngrok.set_auth_token("3497TDD7p62qutWxLVvQHGq3iTo_2PF64hjRKWCwwhNGEyfpK")

In [3]:
# ==================== PAN ANALYZER ====================

import re
from enum import Enum
from dataclasses import dataclass

class TokenType(Enum):
    ADD = 'ADD'
    MIX = 'MIX'
    STIR = 'STIR'
    CHOP = 'CHOP'
    FRY = 'FRY'
    BOIL = 'BOIL'
    BAKE = 'BAKE'
    SERVE = 'SERVE'
    WAIT = 'WAIT'
    SET = 'SET'
    LET = 'LET'
    IF = 'IF'
    ELSE = 'ELSE'
    REPEAT = 'REPEAT'
    TIMES = 'TIMES'
    PROCEDURE = 'PROCEDURE'
    RETURN = 'RETURN'
    PRINT = 'PRINT'
    WHILE = 'WHILE'
    ASSIGN = 'ASSIGN'
    PLUS = 'PLUS'
    MINUS = 'MINUS'
    MULT = 'MULT'
    DIV = 'DIV'
    EQ = 'EQ'
    NE = 'NE'
    LT = 'LT'
    GT = 'GT'
    LE = 'LE'
    GE = 'GE'
    LPAREN = 'LPAREN'
    RPAREN = 'RPAREN'
    LBRACE = 'LBRACE'
    RBRACE = 'RBRACE'
    SEMICOLON = 'SEMICOLON'
    COMMA = 'COMMA'
    ID = 'ID'
    NUMBER = 'NUMBER'
    STRING = 'STRING'
    EOF = 'EOF'
    ERROR = 'ERROR'

@dataclass
class Token:
    type: TokenType
    value: any
    line: int
    col: int

class Lexer:
    def __init__(self, code):
        self.code = code
        self.tokens = []
        self.line = 1
        self.col = 1
        self.keywords = {
            "add": TokenType.ADD, "mix": TokenType.MIX, "stir": TokenType.STIR,
            "chop": TokenType.CHOP, "fry": TokenType.FRY, "boil": TokenType.BOIL,
            "bake": TokenType.BAKE, "serve": TokenType.SERVE, "wait": TokenType.WAIT,
            "set": TokenType.SET, "let": TokenType.LET, "if": TokenType.IF,
            "else": TokenType.ELSE, "repeat": TokenType.REPEAT, "times": TokenType.TIMES,
            "procedure": TokenType.PROCEDURE, "return": TokenType.RETURN,
            "print": TokenType.PRINT, "while": TokenType.WHILE,
        }

    def tokenize(self):
        token_spec = [
            ("NUMBER", r'\d+(\.\d+)?'), ("STRING", r'"[^"\n]*"'),
            ("ID", r'[A-Za-z_][A-Za-z0-9_]*'), ("EQ", r'=='), ("NE", r'!='),
            ("LE", r'<='), ("GE", r'>='), ("ASSIGN", r'='), ("LT", r'<'),
            ("GT", r'>'), ("PLUS", r'\+'), ("MINUS", r'-'), ("MULT", r'\*'),
            ("DIV", r'/'), ("LPAREN", r'\('), ("RPAREN", r'\)'),
            ("LBRACE", r'\{'), ("RBRACE", r'\}'), ("SEMICOLON", r';'),
            ("COMMA", r','), ("NEWLINE", r'\n'), ("SKIP", r'[ \t]+'),
            ("MISMATCH", r'.'),
        ]
        tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_spec)

        for mo in re.finditer(tok_regex, self.code):
            kind = mo.lastgroup
            value = mo.group()
            if kind == "NUMBER":
                tok = Token(TokenType.NUMBER, float(value) if '.' in value else int(value), self.line, self.col)
            elif kind == "STRING":
                tok = Token(TokenType.STRING, value.strip('"'), self.line, self.col)
            elif kind == "ID":
                tok_type = self.keywords.get(value.lower(), TokenType.ID)
                tok = Token(tok_type, value, self.line, self.col)
            elif kind == "NEWLINE":
                self.line += 1
                self.col = 1
                continue
            elif kind == "SKIP":
                self.col += len(value)
                continue
            elif kind == "MISMATCH":
                tok = Token(TokenType.ERROR, value, self.line, self.col)
            else:
                tok = Token(TokenType[kind], value, self.line, self.col)
            self.tokens.append(tok)
            self.col += len(value)

        self.tokens.append(Token(TokenType.EOF, None, self.line, self.col))
        return self.tokens

class ParseNode:
    def __init__(self, node_type: str, children=None, token=None):
        self.type = node_type
        self.children = children or []
        self.token = token

    def to_string(self, depth=0):
        indent = "  " * depth
        result = f"{indent}{'📦' if depth == 0 else '├─'} {self.type}"
        if self.token:
            if self.token.type == TokenType.STRING:
                result += f' 📝 "{self.token.value}"'
            elif self.token.type == TokenType.NUMBER:
                result += f' 🔢 {self.token.value}'
            elif self.token.type in {TokenType.CHOP, TokenType.FRY, TokenType.BOIL,
                                     TokenType.BAKE, TokenType.MIX, TokenType.STIR,
                                     TokenType.SERVE, TokenType.WAIT, TokenType.ADD}:
                result += f' 🍳 {self.token.value}'
            elif self.token.type == TokenType.ID:
                result += f' 🏷️  {self.token.value}'
            else:
                result += f' [{self.token.value}]'
        result += "\n"
        for child in self.children:
            result += child.to_string(depth + 1)
        return result

    def to_html(self, depth=0):
        colors = {
            'procedure_def': '#8b5cf6', 'if_stmt': '#f59e0b', 'while_loop': '#10b981',
            'repeat_loop': '#06b6d4', 'assignment': '#ec4899', 'declaration': '#f43f5e',
            'cook_command': '#84cc16', 'procedure_call': '#6366f1', 'condition': '#f97316',
            'binop': '#a855f7'
        }
        color = colors.get(self.type, '#64748b')
        result = f'<div style="margin-left: {depth*20}px; color: {color}; font-family: monospace;">'
        result += f'<strong>{self.type}</strong>'
        if self.token:
            if self.token.type == TokenType.STRING:
                result += f' <span style="color: #22c55e;">"{self.token.value}"</span>'
            elif self.token.type == TokenType.NUMBER:
                result += f' <span style="color: #3b82f6;">{self.token.value}</span>'
            else:
                result += f' <span style="color: #8b5cf6;">[{self.token.value}]</span>'
        result += '</div>'
        for child in self.children:
            result += child.to_html(depth + 1)
        return result

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.errors = []
        self.in_panic_mode = False

    def current_token(self):
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return Token(TokenType.EOF, None, -1, -1)

    def advance(self):
        if self.pos < len(self.tokens) - 1:
            self.pos += 1
            self.in_panic_mode = False

    def expect(self, token_type):
        if self.current_token().type == token_type:
            return True
        if not self.in_panic_mode:
            self.error(f"Expected {token_type.name} but found {self.current_token().type.name}")
            self.in_panic_mode = True
        return False

    def error(self, message):
        token = self.current_token()
        context = self._get_error_context()
        self.errors.append({
            'line': token.line, 'column': token.col,
            'message': message, 'context': context,
            'suggestion': self._get_suggestion(message)
        })

    def _get_error_context(self):
        context_tokens = []
        start = max(0, self.pos - 2)
        end = min(len(self.tokens), self.pos + 3)
        for i in range(start, end):
            token = self.tokens[i]
            if token.type != TokenType.EOF:
                marker = " >>> " if i == self.pos else " "
                context_tokens.append(f"{marker}{token.value}")
        return " ".join(context_tokens) if context_tokens else None

    def _get_suggestion(self, error_msg):
        prev = self.tokens[self.pos - 1] if self.pos > 0 else None

        if "Expected number or variable after 'repeat'" in error_msg:
            return "Use a number (e.g., 'repeat 3 times') or variable (e.g., 'repeat count times')"

        if "Unexpected closing brace" in error_msg:
            return "Remove this '}' or add matching opening brace before it"
        if "Unexpected opening brace" in error_msg:
            return "Remove this '{' or add a control structure (if/while/repeat/procedure) before it"
        if "Expected SEMICOLON" in error_msg:
            if prev and prev.type == TokenType.ID:
                return f"Add ';' after '{prev.value}'"
            elif prev and prev.type == TokenType.NUMBER:
                return f"Add ';' after the number {prev.value}"
            elif prev and prev.type == TokenType.RPAREN:
                return "Add ';' after the closing parenthesis ')'"
            return "Add semicolon ';' at end of statement"
        if "Expected LPAREN" in error_msg:
            if prev and prev.type == TokenType.WHILE:
                return "Add '(' after 'while' → while (condition)"
            elif prev and prev.type == TokenType.IF:
                return "Add '(' after 'if' → if (condition)"
            elif prev and prev.type == TokenType.ID:
                return f"Add '(' after '{prev.value}' for function call"
            return "Add opening parenthesis '('"
        if "Expected RPAREN" in error_msg:
            return "Add closing parenthesis ')' to match the opening '('"
        if "Expected LBRACE" in error_msg:
            return "Add opening brace '{' to start the block"
        if "Expected RBRACE" in error_msg:
            return "Add closing brace '}' to end the block"
        if "Expected ASSIGN" in error_msg:
            if prev and prev.type == TokenType.ID:
                return f"Add '= value;' after '{prev.value}'"
            return "Add '=' for assignment"
        return "Check syntax near this token"

    def synchronize(self):
        self.in_panic_mode = True
        recovery_tokens = {
            TokenType.SEMICOLON, TokenType.RBRACE, TokenType.PROCEDURE,
            TokenType.IF, TokenType.WHILE, TokenType.REPEAT, TokenType.LET,
            TokenType.SET, TokenType.RETURN, TokenType.PRINT, TokenType.EOF
        }
        while self.current_token().type not in recovery_tokens:
            self.advance()
        if self.current_token().type == TokenType.SEMICOLON:
            self.advance()

    def parse(self):
        node = ParseNode("program")
        while self.current_token().type != TokenType.EOF:
            stmt = self.parse_statement()
            if stmt:
                node.children.append(stmt)
        return node

    def parse_statement(self):
        token = self.current_token()

        # Check for unexpected braces first
        if token.type == TokenType.RBRACE:
            self.error(f"Unexpected closing brace '}}' - no matching opening brace")
            self.advance()
            return None
        elif token.type == TokenType.LBRACE:
            self.error(f"Unexpected opening brace '{{' - must follow a control structure (if, while, repeat, procedure)")
            self.advance()
            return None

        if token.type == TokenType.PROCEDURE:
            return self.parse_procedure_def()
        elif token.type == TokenType.REPEAT:
            return self.parse_repeat_loop()
        elif token.type == TokenType.WHILE:
            return self.parse_while_loop()
        elif token.type == TokenType.IF:
            return self.parse_if_stmt()
        elif token.type == TokenType.LET:
            return self.parse_declaration()
        elif token.type == TokenType.SET:
            return self.parse_assignment()
        elif token.type in {TokenType.CHOP, TokenType.MIX, TokenType.STIR, TokenType.FRY,
                          TokenType.BOIL, TokenType.BAKE, TokenType.SERVE, TokenType.WAIT, TokenType.ADD}:
            return self.parse_cook_command()
        elif token.type == TokenType.PRINT:
            return self.parse_print_stmt()
        elif token.type == TokenType.RETURN:
            node = ParseNode("return_stmt", token=token)
            self.advance()
            if not self.expect(TokenType.SEMICOLON):
                return node
            self.advance()
            return node
        elif token.type == TokenType.ID:
            return self.parse_procedure_call()
        else:
            self.error(f"Unexpected token {token.type.name}")
            self.synchronize()
            return ParseNode("error")

    def parse_declaration(self):
        node = ParseNode("declaration")
        self.advance()
        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected variable name after 'let'")
            return node
        if self.current_token().type == TokenType.ASSIGN:
            node.children.append(ParseNode("assign", token=self.current_token()))
            self.advance()
            node.children.append(self.parse_expr())
        else:
            self.error("Expected '=' in declaration")
            return node
        if not self.expect(TokenType.SEMICOLON):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("semicolon", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        return node

    def parse_assignment(self):
        node = ParseNode("assignment")
        self.advance()
        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected variable name after 'set'")
            return node
        if not self.expect(TokenType.ASSIGN):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("assign", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_expr())
        if not self.expect(TokenType.SEMICOLON):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("semicolon", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        return node

    def parse_while_loop(self):
        node = ParseNode("while_loop", token=self.current_token())
        self.advance()
        if not self.expect(TokenType.LPAREN):
            pass
        else:
            node.children.append(ParseNode("lparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_condition())
        if not self.expect(TokenType.RPAREN):
            pass
        else:
            node.children.append(ParseNode("rparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.LBRACE):
            pass
        else:
            node.children.append(ParseNode("lbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_statement_list())
        if not self.expect(TokenType.RBRACE):
            pass
        else:
            node.children.append(ParseNode("rbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        return node

    def parse_repeat_loop(self):
        node = ParseNode("repeat_loop")
        self.advance()
        if self.current_token().type == TokenType.NUMBER:
            node.children.append(ParseNode("number", token=self.current_token()))
            self.advance()
        elif self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("variable", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected number or variable after 'repeat'")
        if not self.expect(TokenType.TIMES):
            pass
        else:
            node.children.append(ParseNode("times", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.LBRACE):
            pass
        else:
            node.children.append(ParseNode("lbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_statement_list())
        if not self.expect(TokenType.RBRACE):
            pass
        else:
            node.children.append(ParseNode("rbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        return node

    def parse_if_stmt(self):
        node = ParseNode("if_stmt")
        self.advance()
        if not self.expect(TokenType.LPAREN):
            pass
        else:
            node.children.append(ParseNode("lparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_condition())
        if not self.expect(TokenType.RPAREN):
            pass
        else:
            node.children.append(ParseNode("rparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.LBRACE):
            pass
        else:
            node.children.append(ParseNode("lbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_statement_list())
        if not self.expect(TokenType.RBRACE):
            pass
        else:
            node.children.append(ParseNode("rbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        if self.current_token().type == TokenType.ELSE:
            node.children.append(ParseNode("else", token=self.current_token()))
            self.advance()
            if not self.expect(TokenType.LBRACE):
                pass
            else:
                node.children.append(ParseNode("lbrace", token=self.current_token()))
                self.advance()
            self.in_panic_mode = False
            node.children.append(ParseNode("else_block", children=[self.parse_statement_list()]))
            if not self.expect(TokenType.RBRACE):
                pass
            else:
                node.children.append(ParseNode("rbrace", token=self.current_token()))
                self.advance()
            self.in_panic_mode = False
        return node

    def parse_procedure_def(self):
        node = ParseNode("procedure_def")
        self.advance()
        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("id", token=self.current_token()))
            self.advance()
        else:
            self.error("Expected procedure name")
        if not self.expect(TokenType.LPAREN):
            pass
        else:
            node.children.append(ParseNode("lparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        params = ParseNode("params")
        while self.current_token().type == TokenType.ID:
            params.children.append(ParseNode("param", token=self.current_token()))
            self.advance()
            if self.current_token().type == TokenType.COMMA:
                params.children.append(ParseNode("comma", token=self.current_token()))
                self.advance()
            else:
                break
        node.children.append(params)
        if not self.expect(TokenType.RPAREN):
            pass
        else:
            node.children.append(ParseNode("rparen", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.LBRACE):
            pass
        else:
            node.children.append(ParseNode("lbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_statement_list())
        if not self.expect(TokenType.RBRACE):
            pass
        else:
            node.children.append(ParseNode("rbrace", token=self.current_token()))
            self.advance()
        self.in_panic_mode = False
        return node

    def parse_procedure_call(self):
        node = ParseNode("procedure_call")
        node.children.append(ParseNode("id", token=self.current_token()))
        self.advance()
        if not self.expect(TokenType.LPAREN):
            while self.current_token().type not in {TokenType.SEMICOLON, TokenType.RBRACE, TokenType.EOF}:
                self.advance()
            if self.current_token().type == TokenType.SEMICOLON:
                node.children.append(ParseNode("semicolon", token=self.current_token()))
                self.advance()
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("lparen", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        args = ParseNode("args")
        if self.current_token().type != TokenType.RPAREN:
            args.children.append(self.parse_expr())
            while self.current_token().type == TokenType.COMMA:
                args.children.append(ParseNode("comma", token=self.current_token()))
                self.advance()
                args.children.append(self.parse_expr())
        node.children.append(args)
        if not self.expect(TokenType.RPAREN):
            while self.current_token().type not in {TokenType.SEMICOLON, TokenType.RBRACE, TokenType.EOF}:
                self.advance()
            if self.current_token().type == TokenType.SEMICOLON:
                node.children.append(ParseNode("semicolon", token=self.current_token()))
                self.advance()
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("rparen", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.SEMICOLON):
            if self.current_token().type == TokenType.SEMICOLON:
                node.children.append(ParseNode("semicolon", token=self.current_token()))
                self.advance()
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("semicolon", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        return node

    def parse_cook_command(self):
        node = ParseNode("cook_command", token=self.current_token())
        self.advance()
        if self.current_token().type == TokenType.ID:
            node.children.append(ParseNode("ingredient", token=self.current_token()))
            self.advance()
        if self.current_token().type == TokenType.NUMBER:
            node.children.append(ParseNode("time", token=self.current_token()))
            self.advance()
        if not self.expect(TokenType.SEMICOLON):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("semicolon", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        return node

    def parse_print_stmt(self):
        node = ParseNode("print_stmt")
        self.advance()
        if not self.expect(TokenType.LPAREN):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("lparen", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        node.children.append(self.parse_expr())
        if not self.expect(TokenType.RPAREN):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("rparen", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        if not self.expect(TokenType.SEMICOLON):
            self.in_panic_mode = False
            return node
        node.children.append(ParseNode("semicolon", token=self.current_token()))
        self.advance()
        self.in_panic_mode = False
        return node

    def parse_statement_list(self):
        node = ParseNode("statement_list")
        while self.current_token().type not in {TokenType.RBRACE, TokenType.EOF}:
            if self.in_panic_mode:
                self.synchronize()
                continue
            stmt = self.parse_statement()
            if stmt and stmt.type != "error":
                node.children.append(stmt)
        return node

    def parse_condition(self):
        node = ParseNode("condition")
        node.children.append(self.parse_expr())
        if self.current_token().type in {TokenType.EQ, TokenType.NE, TokenType.LT,
                                        TokenType.GT, TokenType.LE, TokenType.GE}:
            node.children.append(ParseNode("operator", token=self.current_token()))
            self.advance()
            node.children.append(self.parse_expr())
        return node

    def parse_expr(self):
        node = self.parse_term()
        while self.current_token().type in {TokenType.PLUS, TokenType.MINUS}:
            op = ParseNode("operator", token=self.current_token())
            self.advance()
            right = self.parse_term()
            node = ParseNode("binop", children=[node, op, right])
        return node

    def parse_term(self):
        node = self.parse_factor()
        while self.current_token().type in {TokenType.MULT, TokenType.DIV}:
            op = ParseNode("operator", token=self.current_token())
            self.advance()
            right = self.parse_factor()
            node = ParseNode("binop", children=[node, op, right])
        return node

    def parse_factor(self):
        token = self.current_token()
        if token.type == TokenType.NUMBER:
            node = ParseNode("number", token=token)
            self.advance()
            return node
        elif token.type == TokenType.STRING:
            node = ParseNode("string", token=token)
            self.advance()
            return node
        elif token.type == TokenType.ID:
            node = ParseNode("id", token=token)
            self.advance()
            return node
        elif token.type == TokenType.LPAREN:
            lparen_node = ParseNode("lparen", token=token)
            self.advance()
            expr_node = self.parse_expr()
            if not self.expect(TokenType.RPAREN):
                self.in_panic_mode = False
                return ParseNode("grouped_expr", children=[lparen_node, expr_node])
            rparen_node = ParseNode("rparen", token=self.current_token())
            self.advance()
            self.in_panic_mode = False
            return ParseNode("grouped_expr", children=[lparen_node, expr_node, rparen_node])
        else:
            self.error(f"Unexpected token {token.type.name}")
            self.in_panic_mode = False
            return ParseNode("error")

# ==================== FLASK APP ====================
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    return '''<!DOCTYPE html>
<html><head><meta charset="UTF-8"><title>PAN Analyzer</title>
<style>
* { margin: 0; padding: 0; box-sizing: border-box; }
body { font-family: 'Segoe UI', sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
.container { max-width: 1600px; margin: 0 auto; background: white; border-radius: 15px; box-shadow: 0 20px 60px rgba(0,0,0,0.3); overflow: hidden; display: flex; flex-direction: column; min-height: 90vh; }
.header { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; text-align: center; }
.header h1 { font-size: 2.5em; margin-bottom: 10px; }
.main-content { display: flex; flex: 1; gap: 20px; padding: 20px; }
.editor-section, .output-section { display: flex; flex-direction: column; flex: 1; background: #f8f9fa; border-radius: 10px; border: 2px solid #e9ecef; }
.section-title { background: #667eea; color: white; padding: 15px 20px; font-weight: bold; }
.editor-wrapper { display: flex; flex: 1; }
.line-numbers { background: #2d2d2d; color: #858585; padding: 15px 10px; font-family: 'Courier New', monospace; font-size: 14px; text-align: right; min-width: 50px; border-right: 2px solid #667eea; line-height: 1.5; white-space: pre; }
textarea { flex: 1; padding: 15px; border: none; font-family: 'Courier New', monospace; font-size: 14px; resize: none; line-height: 1.5; }
textarea:focus { outline: none; background: #f0f4ff; }
.controls { display: flex; gap: 10px; padding: 15px; background: white; border-top: 2px solid #e9ecef; }
button { padding: 12px 25px; font-size: 1em; border: none; border-radius: 8px; cursor: pointer; font-weight: bold; transition: all 0.3s; }
.btn-analyze { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; flex: 1; }
.btn-analyze:hover { box-shadow: 0 10px 20px rgba(102,126,234,0.4); transform: translateY(-2px); }
.btn-clear { background: #e9ecef; }
.btn-clear:hover { background: #dee2e6; }
.btn-example { background: #28a745; color: white; }
.btn-example:hover { background: #218838; }
.output-content { flex: 1; overflow-y: auto; padding: 20px; background: white; }
.message { margin-bottom: 15px; padding: 15px; border-radius: 8px; border-left: 5px solid; font-size: 13px; }
.success { background: #d4edda; border-color: #28a745; color: #155724; }
.error { background: #f8d7da; border-color: #dc3545; color: #721c24; }
.error .context { background: #fff; padding: 8px; margin-top: 8px; border-radius: 4px; font-family: monospace; font-size: 11px; }
.error .suggestion { margin-top: 8px; color: #856404; background: #fff3cd; padding: 6px; border-radius: 4px; font-size: 11px; }
.status-badge { display: inline-block; padding: 8px 15px; border-radius: 20px; font-weight: bold; margin-bottom: 15px; }
.status-success { background: #d4edda; color: #155724; }
.status-error { background: #f8d7da; color: #721c24; }
.section-subtitle { font-weight: bold; color: #667eea; margin-top: 15px; margin-bottom: 8px; font-size: 14px; }
.token-list { background: #f8f9fa; padding: 10px; border-radius: 5px; margin-bottom: 15px; font-size: 12px; max-height: 250px; overflow-y: auto; }
.token-item { padding: 5px 8px; margin: 3px 0; background: white; border-left: 3px solid #667eea; border-radius: 3px; }
.tree-container { background: #1e1e1e; color: #d4d4d4; padding: 15px; border-radius: 5px; font-size: 12px; max-height: 400px; overflow-y: auto; white-space: pre; font-family: 'Courier New', monospace; line-height: 1.6; }
.stats { display: flex; gap: 15px; margin-bottom: 15px; }
.stat-box { flex: 1; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px; border-radius: 8px; text-align: center; }
.stat-box .number { font-size: 2em; font-weight: bold; }
.stat-box .label { font-size: 0.9em; opacity: 0.9; }
</style></head><body>
<div class="container">
<div class="header"><h1>🍳 PAN Syntax Analyzer</h1><p>Programming Languages Project</p></div>
<div class="main-content">
<div class="editor-section">
<div class="section-title">📝 Code Editor</div>
<div class="editor-wrapper">
<div class="line-numbers" id="lineNumbers">1</div>
<textarea id="codeInput" spellcheck="false">chop gulay;
fry gulay 10;
mix;
serve;</textarea>
</div>
<div class="controls">
<button class="btn-analyze" onclick="analyzeCode()">▶ Analyze Code</button>
<button class="btn-clear" onclick="clearCode()">🗑 Clear</button>
<button class="btn-example" onclick="loadExample()">📚 Random Example</button>
</div>
</div>
<div class="output-section">
<div class="section-title">📊 Analysis Results</div>
<div class="output-content" id="outputContent">
<p style="color: #999; text-align: center; margin-top: 50px;">👈 Click "Analyze Code" to see results</p>
</div>
</div>
</div>
</div>
<script>
const codeInput = document.getElementById('codeInput');
const lineNumbers = document.getElementById('lineNumbers');

function updateLineNumbers() {
    const lines = codeInput.value.split('\\n');
    const numbers = lines.map((_, i) => i + 1).join('\\n');
    lineNumbers.textContent = numbers;
}

codeInput.addEventListener('input', updateLineNumbers);
codeInput.addEventListener('scroll', () => {
    lineNumbers.scrollTop = codeInput.scrollTop;
});

updateLineNumbers();

async function analyzeCode() {
    const code = codeInput.value;
    const output = document.getElementById('outputContent');

    if (!code.trim()) {
        output.innerHTML = '<div class="message error">⚠️ Please enter code</div>';
        return;
    }

    output.innerHTML = '<p style="color: #999; text-align: center; margin-top: 50px;">⏳ Analyzing...</p>';

    try {
        const response = await fetch('/analyze', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({code: code})
        });

        if (!response.ok) {
            throw new Error('Server returned error: ' + response.status);
        }

        const data = await response.json();

        let html = '';
        const hasErrors = data.errors.length > 0;
        const statusClass = hasErrors ? 'status-error' : 'status-success';
        const statusText = hasErrors ? '❌ ERRORS FOUND' : '✅ SYNTAX VALID';
        html += `<div class="status-badge ${statusClass}">${statusText}</div>`;

        html += '<div class="stats">';
        html += `<div class="stat-box"><div class="number">${data.tokens.length}</div><div class="label">Tokens</div></div>`;
        html += `<div class="stat-box"><div class="number">${data.errors.length}</div><div class="label">Errors</div></div>`;
        html += `<div class="stat-box"><div class="number">${code.split('\\n').length}</div><div class="label">Lines</div></div>`;
        html += '</div>';

        if (hasErrors) {
            html += `<div class="section-subtitle">🚨 Error Details (${data.errors.length} found)</div>`;
            data.errors.forEach((err, idx) => {
                html += `<div class="message error">`;
                html += `<strong>Error #${idx + 1}</strong> at Line ${err.line}, Column ${err.column}<br>`;
                html += `📍 ${err.message}`;
                if (err.context) {
                    html += `<div class="context">Context: ${escapeHtml(err.context)}</div>`;
                }
                if (err.suggestion) {
                    html += `<div class="suggestion">💡 ${err.suggestion}</div>`;
                }
                html += `</div>`;
            });
        } else {
            html += `<div class="message success"><strong>✓ Perfect! No errors found. Walang problema!</strong></div>`;
        }

        html += '<div class="section-subtitle">📋 Token Stream</div><div class="token-list">';
        data.tokens.forEach(t => {
            html += `<div class="token-item"><strong>${t.type}</strong> → ${escapeHtml(String(t.value))} <span style="color: #999;">(L${t.line}:C${t.col})</span></div>`;
        });
        html += '</div>';

        if (data.parse_tree_html && data.parse_tree_html.trim()) {
            html += '<div class="section-subtitle">🌳 Parse Tree</div><div class="tree-container">' + data.parse_tree_html + '</div>';
        } else if (data.parse_tree && data.parse_tree.trim()) {
            html += '<div class="section-subtitle">🌳 Parse Tree</div><div class="tree-container">' + escapeHtml(data.parse_tree) + '</div>';
        }

        output.innerHTML = html;
    } catch(e) {
        console.error('Error during analysis:', e);
        output.innerHTML = `<div class="message error">❌ Connection Error: ${e.message}<br><br>Check browser console (F12) for details.</div>`;
    }
}

function clearCode() {
    codeInput.value = '';
    updateLineNumbers();
    document.getElementById('outputContent').innerHTML = '<p style="color: #999; text-align: center; margin-top: 50px;">👈 Click "Analyze Code" to see results</p>';
}

function loadExample() {
    const examples = [
        `add rice;
boil rice 15;
mix;
serve;`,

`procedure gawaKape() {
    boil tubig 3;
    add kape;
    stir;
}
gawaKape();
serve;`,

`if (init > 300) {
    print("Tamang init!");
    fry isda 5;
} else {
    print("Painitin pa!");
    wait 2;
}
serve;`,

`repeat 4 times {
    mix;
    wait 1;
}
serve;`,

`procedure luto(itlog) {
    fry itlog 5;
    mix;
    serve;
}
if (gulay == fresh) {
    luto(itlog);
} else {
    boil itlog;
}`,

`taste sabaw;
if (maalat) {
    add tubig;
} else {
    add asin;
}
mix;
serve;`,

`repeat 3 times {
    stir;
    print("Hinahalo pa...");
}
serve;`,

`procedure prituhin(sangkap) {
    heat kawali;
    add mantika 3;
    fry sangkap 5;
}
prituhin(manok);
serve;`,

`repeat 5 times {
    print("Naghihintay...");
    wait 1;
}
bake cake 20;
serve;`,

`taste adobo;
if (kulang_sa_toyo) {
    add toyo 1;
} else {
    add suka 1;
}
mix;
serve;`,

`repeat 2 times {
    chop gulay;
    repeat 3 times {
        stir;
    }
}
serve;`,

`procedure timpla(inumin) {
    add gatas;
    add asukal;
    stir;
}
timpla(gatas);
serve;`,

`if (ulam == adobo) {
    fry baboy 10;
} else {
    if (ulam == sinigang) {
        boil baboy 15;
    } else {
        grill isda;
    }
}
serve;`,

`repeat 5 times {
    taste sabaw;
    if (kulang_sa_asin) {
        add asin 1;
    }
}
serve;`,

`procedure gawaDessert(item) {
    chop item;
    add gatas;
    mix;
    chill 10;
}
gawaDessert(saging);
serve;`,

`if (temp < 200) {
    increase_heat;
} else {
    print("Tamang init na");
}
bake tinapay 15;
serve;`,

`add harina;
add itlog;
add asukal;
mix;
bake cake 20;
serve;`,

`procedure tikim() {
    taste sabaw;
    print("Tikim...");
}
repeat 3 times {
    tikim();
}
serve;`,

`if (ulam == manok) {
    fry manok 10;
} else {
    boil isda 8;
}
mix;
serve;`,

`procedure lutuin(sangkap) {
    repeat 2 times {
        chop sangkap;
        fry sangkap 5;
    }
    mix;
}
if (count > 2) {
    lutuin(gulay);
} else {
    boil gulay;
}
serve;`
];
    codeInput.value = examples[Math.floor(Math.random() * examples.length)];
    updateLineNumbers();
    setTimeout(analyzeCode, 100);
}

function escapeHtml(text) {
    const map = {'&': '&amp;', '<': '&lt;', '>': '&gt;', '"': '&quot;', "'": '&#039;'};
    return text.replace(/[&<>"']/g, m => map[m]);
}
</script>
</body></html>'''

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    code = data.get('code', '')

    try:
        lexer = Lexer(code)
        tokens = lexer.tokenize()
        parser = Parser(tokens)
        tree = parser.parse()

        result = {
            'tokens': [{'type': t.type.value, 'value': str(t.value), 'line': t.line, 'col': t.col} for t in tokens[:-1]],
            'errors': parser.errors,
            'parse_tree': tree.to_string(),
            'parse_tree_html': tree.to_html(),
            'status': 'valid' if not parser.errors else 'error'
        }
        return jsonify(result)
    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({
            'tokens': [],
            'errors': [{'line': -1, 'column': -1, 'message': f'Internal error: {str(e)}', 'context': None, 'suggestion': 'Check server logs'}],
            'parse_tree': '',
            'parse_tree_html': '',
            'status': 'error'
        }), 200

# ==================== RUN SERVER ====================
from pyngrok import ngrok

try:
    ngrok.kill()
except:
    pass

ngrok.set_auth_token("3497TDD7p62qutWxLVvQHGq3iTo_2PF64hjRKWCwwhNGEyfpK")
public_url = ngrok.connect(5000)

print(f"\n{'='*70}")
print(f"✅ PAN ANALYZER IS LIVE! (FIXED VERSION)")
print(f"{'='*70}")
print(f"🌐 URL: {public_url}")
print(f"{'='*70}")

app.run(port=5000, debug=False)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://mycelial-ronan-vagabondish.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}
